imports

In [31]:
import random
from datetime import datetime,timedelta
import csv
from faker import Faker

## Goal: generate a list of task with the people on it and the deadline of each task

### First let's just try to determine the duration of a task

As no dataset lets generate our own one

In [32]:
#task : desc,type,deadline,technologies,team,duration(the output that we try to guess)
#start: date of the task creation
#type : "bug", "feature", "refactor", "test", "documentation"
#technologies : "python", "java", "c++", "c#", "javascript", "ruby", "php", "html", "css", "sql", "nosql"
#team : number of people in the team
#duration : in days
#deadline : a date in the future
#desc : a string

#generate a random but credible dataset of task

def generate_dataset():
    
    fake = Faker()
    tasks = []
    for i in range(1000):
        min_duration = 1
        task = {}
        task["desc"] = fake.text()
        task["type"] = random.choice(["bug", "feature", "refactor", "test", "documentation"])
        task["start"] = datetime.now().strftime("%Y-%m-%d")
        task["technologies"] = random.choices(["python", "java", "c++", "c#", "javascript", "ruby", "php", "html", "css", "sql", "nosql"], k=random.randint(1, 3))
        task["team"] = random.randint(1, 10)
        if task["team"]< len(task["technologies"]):
            min_duration += len(task["technologies"]) - task["team"] #case more technologies than team members
        if "c++" in task["technologies"]:
            min_duration += 1 #because c++ you know...
        if "documentation" not in task["type"]:
            min_duration += 1 #documentation is easier
        
        if len(task["desc"]) > 100:
            min_duration += len(task["desc"])//100 #we can imagine than longer description = more time
        
        #task["deadline"] should be at least min_duration days in the future
        start_date = datetime.now()
        min_deadline_date = start_date + timedelta(days=min_duration)
        max_duration = 45
        max_deadline_date = start_date + timedelta(days=max_duration)
        task["deadline"] = fake.date_time_between_dates(datetime_start=min_deadline_date, datetime_end=max_deadline_date).strftime("%Y-%m-%d")
        task["duration"] = random.randint(min_duration, max_duration)
        tasks.append(task)
    with open("./data/tasks.csv", "w") as f:
        writer = csv.DictWriter(f, fieldnames=["desc", "type", "start","deadline", "technologies", "team", "duration"])
        writer.writeheader()
        for task in tasks:
            writer.writerow(task)
    return tasks

generate_dataset()

[{'desc': 'Participant baby game what. Choice recognize soldier mind detail manager weight southern. Field husband seat measure take cost future matter. Discussion person fact Congress sense kid.',
  'type': 'feature',
  'start': '2024-11-10',
  'technologies': ['php'],
  'team': 1,
  'deadline': '2024-12-05',
  'duration': 20},
 {'desc': 'Remember budget quality trouble. Organization adult building article. Realize size practice end agent support around.',
  'type': 'feature',
  'start': '2024-11-10',
  'technologies': ['c++'],
  'team': 9,
  'deadline': '2024-12-04',
  'duration': 8},
 {'desc': 'American degree threat season yet.\nBecome ground scene special brother.\nFinally lose soldier real share upon. Lay away crime reality drive left now.',
  'type': 'documentation',
  'start': '2024-11-10',
  'technologies': ['css'],
  'team': 7,
  'deadline': '2024-11-20',
  'duration': 6},
 {'desc': 'Large yourself pattern teach method leader who. When every ground wife.\nGlass born key wonde